# TWC data requests

We demonstrate three data requests from TWC using the TWC 
core package. Documentation on the package available [here](https://docs.google.com/document/d/1TPj7h6z4Gjaqjlup4rJ8B43Ge7n6iXfCF8GLw74lOeA/edit)
1. Daily forecasts (from today) for 3, 5, 7, 10, and 15 
day ahead (user selects the desired forecast horizon). Documentation available [here](https://ibm.co/V3DFap)

2. Hourly forecasts (from today) 2 days ahead.. 10-day 
ahead forecasts are available with Enhanced forecast API. Documentation available [here](https://ibm.co/v3HFap)
3. Historical data from the `History on Demand` package
for user defined start - end dates. Documentation available [here](https://docs.google.com/document/d/12V8zpVRh15cgP0IiuQ6n875-I04_rl2J28uE0jQcblI/edit)

# API key

In [1]:
api_key = 'be01c34853ca4cae81c34853cabcaef1'

/Users/fearghalodonncha/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import requests
import json
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime as dt, timedelta, date
import argparse
from pathlib import Path

In [3]:
def get_twc_forecast_daily(latitude: float, longitude: float, days: int, api_key: str, save_to_file=False):
    '''
    Parameters:
        longitude (float):  point coordinates  
        latitude (float):   of lat and long
        days (int): number of days forecast from today
            TWC core package provides forecast for 3, 5, 7, 
            10, and 15 day forecasts. If we select values 
            outside these API will not return data    
    '''
    assert days in [3, 5, 7, 10, 15], "available forecast windows are 3, 5, 7, 10, and 15 days"
    geocode = '{},{}'.format(latitude, longitude)
    url = 'https://api.weather.com/v3/wx/forecast/daily/{}day?geocode={}&format=json&units=e&language=en-US&apiKey={}'.format(days, geocode, api_key)
    try:
        r = requests.get(url)
        forecast = json.loads(r.text)
        forecast.pop('daypart')
        json.dumps(forecast, indent=1)
        df = pd.DataFrame.from_dict(forecast)
        df.set_index('validTimeUtc', inplace=True)
        df.index = [dt.utcfromtimestamp(x) for x in df.index]
        df = df.drop_duplicates()
        if save_to_file:
             # Create WeatherData folder if doesn't exist
            Path("./WeatherData").mkdir(parents=True, exist_ok=True)
            fname_out = 'WeatherData/TWC_forecast_data_Full_lat' + str(latitude) \
              + '_lon' + str(longitude)+ '_' +  str(days) + "days_from_"  + date.today().strftime("%Y-%m-%d") + '.csv'
            df.to_csv(fname_out)
    except ValueError:
        print('Decoding Failed for lat '  + str(latitude) \
              + ' lon ' + str(longitude)+ ' ' +  str(days) + " days from "  + date.today().strftime("%Y-%m-%d"))
        return  -999, -999
    return df


In [4]:
latitude = 34.0239524929432       # Take USC coordinates as demo
longitude = -118.2857607301622
days = 10 ## number of days ahead to extract forecast values
# Note enhanced package required to access 15 day forecast
# request forecast values for next 10 day. Note if wish to save to file, then set save_to_file = True
df = get_twc_forecast_daily(latitude, longitude, days, api_key, save_to_file=False)
df.head()

,calendarDayTemperatureMax,calendarDayTemperatureMin,dayOfWeek,expirationTimeUtc,moonPhase,moonPhaseCode,moonPhaseDay,moonriseTimeLocal,moonriseTimeUtc,moonsetTimeLocal,...,narrative,qpf,qpfSnow,sunriseTimeLocal,sunriseTimeUtc,sunsetTimeLocal,sunsetTimeUtc,temperatureMax,temperatureMin,validTimeLocal
2021-08-20 14:00:00,78,65,Friday,1629470936,Waxing Gibbous,WXG,13,2021-08-20T18:53:49-0700,1.629511e+09,2021-08-20T04:05:17-0700,...,Partly cloudy. Highs in the upper 70s and lows...,0.0,0.0,2021-08-20T06:18:50-0700,1629465530,2021-08-20T19:33:29-0700,1629513209,78,66,2021-08-20T07:00:00-0700
2021-08-21 14:00:00,76,65,Saturday,1629470936,Full Moon,F,14,2021-08-21T19:34:55-0700,1.629600e+09,2021-08-21T05:13:18-0700,...,More sun than clouds. Highs in the mid 70s and...,0.0,0.0,2021-08-21T06:19:33-0700,1629551973,2021-08-21T19:32:16-0700,1629599536,76,65,2021-08-21T07:00:00-0700
2021-08-22 14:00:00,77,63,Sunday,1629470936,Full Moon,F,15,2021-08-22T20:10:37-0700,1.629688e+09,2021-08-22T06:20:52-0700,...,Mix of sun and clouds. Highs in the upper 70s ...,0.0,0.0,2021-08-22T06:20:15-0700,1629638415,2021-08-22T19:31:02-0700,1629685862,77,63,2021-08-22T07:00:00-0700
2021-08-23 14:00:00,77,63,Monday,1629470936,Waning Gibbous,WNG,16,2021-08-23T20:41:48-0700,1.629777e+09,2021-08-23T07:24:59-0700,...,Partly cloudy. Highs in the upper 70s and lows...,0.0,0.0,2021-08-23T06:20:58-0700,1629724858,2021-08-23T19:29:47-0700,1629772187,77,62,2021-08-23T07:00:00-0700
2021-08-24 14:00:00,82,62,Tuesday,1629470936,Waning Gibbous,WNG,17,2021-08-24T21:09:49-0700,1.629865e+09,2021-08-24T08:27:11-0700,...,Abundant sunshine. Highs in the low 80s and lo...,0.0,0.0,2021-08-24T06:21:41-0700,1629811301,2021-08-24T19:28:32-0700,1629858512,82,65,2021-08-24T07:00:00-0700


In [5]:
def get_twc_forecast_hourly(latitude: float, longitude: float, days: int, api_key: str, save_to_file=False):
    '''
    The hourly forecast values are arguably more focused on 
    industry operations. Returns pandas dataframe of meteorological
    variables such as temperature, wind speed, humidity.
    
    Note that TWC core package provides 2 day forecasts of hourly values
    Enhanced forecast package provides 15 day forecast. Need to ensure
    that your specific API key is valid to request 15 day forecast.
    Parameters:
        longitude (float):  point coordinates  
        latitude (float):   of lat and long
        days (int): number of days forecast from today, 2 or 15 (enhanced) 
    '''
    assert days in [2, 15], "available forecast windows are 2 or 15 days"
    geocode = '{},{}'.format(latitude, longitude)
    url = 'https://api.weather.com/v3/wx/forecast/hourly/{}day?geocode={}&format=json&units=e&language=en-US&apiKey={}'.format(days, geocode, api_key)
    try:
        r = requests.get(url)
        forecast = json.loads(r.text)
        json.dumps(forecast, indent=1)
        df = pd.DataFrame.from_dict(forecast)
        df.set_index('validTimeUtc', inplace=True)
        df.index = [dt.utcfromtimestamp(x) for x in df.index]
        df = df.drop_duplicates()
        if save_to_file:
            Path("./WeatherData").mkdir(parents=True, exist_ok=True)
            fname_out = 'WeatherData/TWC_forecast_data_Full_lat' + str(latitude) \
              + '_lon' + str(longitude)+ '_' +  str(days) + "days_from_"  + date.today().strftime("%Y-%m-%d") + '.csv'
            df.to_csv(fname_out)
    except ValueError:
        print('Decoding Failed for lat '  + str(latitude) \
              + ' lon ' + str(longitude)+ ' ' +  str(days) + " days from "  + date.today().strftime("%Y-%m-%d"))
        return  -999, -999
    return df

In [6]:
## In equivalent manner to above, we can request hourly values. Here we can 
## request 2 day forecast while enhanced package provides 15 day forecast
days = 2
df_h = get_twc_forecast_hourly(latitude, longitude, days, api_key)
df_h.head()

,cloudCover,dayOfWeek,dayOrNight,expirationTimeUtc,iconCode,iconCodeExtend,precipChance,precipType,pressureMeanSeaLevel,qpf,...,uvIndex,validTimeLocal,visibility,windDirection,windDirectionCardinal,windGust,windSpeed,wxPhraseLong,wxPhraseShort,wxSeverity
2021-08-20 15:00:00,42,Friday,D,1629470807,30,3000,2,rain,29.87,0.0,...,1,2021-08-20T08:00:00-0700,10.0,242,WSW,None,1,Partly Cloudy,P Cloudy,1
2021-08-20 16:00:00,40,Friday,D,1629470807,30,3000,2,rain,29.87,0.0,...,2,2021-08-20T09:00:00-0700,10.0,229,SW,None,1,Partly Cloudy,P Cloudy,1
2021-08-20 17:00:00,34,Friday,D,1629470807,30,3000,1,rain,29.86,0.0,...,5,2021-08-20T10:00:00-0700,10.0,224,SW,None,1,Partly Cloudy,P Cloudy,1
2021-08-20 18:00:00,25,Friday,D,1629470807,34,3400,1,rain,29.86,0.0,...,7,2021-08-20T11:00:00-0700,10.0,227,SW,None,2,Mostly Sunny,M Sunny,1
2021-08-20 19:00:00,19,Friday,D,1629470807,32,3200,1,rain,29.84,0.0,...,9,2021-08-20T12:00:00-0700,10.0,233,SW,None,2,Sunny,Sunny,1


In [7]:
def get_twc_history_on_demand(latitude: float, longitude: float, start_date,
                              end_date, api_key, save_to_file=False):
    '''
    Parameters:
        longitude (float):  point coordinates  
        latitude (float):   of lat and long
        start_date (timestamp): starting datetime of data request
        end_date (timestamp): ending datetime of data request  
    '''
    twc_date_begin = start_date.strftime("%Y%m%d%H%M")
    twc_date_end = end_date.strftime("%Y%m%d%H%M")
    geocode = '{},{}'.format(latitude, longitude)
    pointType = 'weighted' # see documentation for more details on this selection
    url = 'https://api.weather.com/v3/wx/hod/reanalysis/historical/point?pointType={}&geocode={}&distance=50&startDateTime={}&endDateTime={}&units=m&format=json&apiKey={}'.format(
        pointType, geocode, twc_date_begin, twc_date_end, api_key)
    try:
        r = requests.get(url)
        forecast = json.loads(r.text)
        json.dumps(forecast, indent=1)
        df = pd.DataFrame.from_dict(forecast)
        df.set_index('observationTimeUtcIso', inplace=True)
        df = df.drop_duplicates()
        
        if save_to_file:
             # Create WeatherData folder if doesn't exist
            Path("./WeatherData").mkdir(parents=True, exist_ok=True)
            fname_out = 'WeatherData/TWC_history_on_demand_lat' + str(latitude) \
              + '_lon' + str(longitude)+ '_'  + date.today().strftime("%Y-%m-%d") + '.csv'
            df.to_csv(fname_out)
    except ValueError:
        print('Decoding Failed for lat '  + str(latitude) \
              + ' lon ' + str(longitude)+ ' ' +  str(days) + " days from "  + date.today().strftime("%Y-%m-%d"))
        print(r.text)
    return df


### Finally we request historical data from the `History on Demand` package
We request one week of data from January 2021

In [8]:

start_date = dt(2021,1,1)
end_date = dt(2021,1,7)
df_hod = get_twc_history_on_demand(30, -10, start_date, end_date, api_key)
df_hod.head()

https://api.weather.com/v3/wx/hod/reanalysis/historical/point?pointType=weighted&geocode=30,-10&distance=50&startDateTime=202101010000&endDateTime=202101070000&units=m&format=json&apiKey=be01c34853ca4cae81c34853cabcaef1


,temperature,dewpoint,relativeHumidity,pressure,windSpeed,windDirection,precipitationRate,latitude,longitude
observationTimeUtcIso,,,,,,,,,
2021-01-01T00:00:00+0000,16.9,10.0,63.7,1018.7,11.7,4.6,0.0,30.0,-10.0
2021-01-01T01:00:00+0000,16.8,10.1,65.0,1018.3,16.8,11.8,0.0,30.0,-10.0
2021-01-01T02:00:00+0000,16.5,9.7,64.2,1018.4,17.6,13.4,0.0,30.0,-10.0
2021-01-01T03:00:00+0000,16.3,9.3,63.3,1018.4,16.7,12.3,0.0,30.0,-10.0
2021-01-01T04:00:00+0000,16.1,8.8,62.0,1018.3,16.9,10.1,0.0,30.0,-10.0
